# Settings

In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/vidie-audie

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/vidie-audie


In [1]:
%cd /content/drive/MyDrive/vidie-audie

/content/drive/MyDrive/vidie-audie


In [4]:
# env loading
!pip install python-dotenv

from dotenv import load_dotenv
import os

dotenv_path = '/content/drive/MyDrive/vidie-audie/.env'

load_dotenv(dotenv_path)
ngrok_token = os.getenv('NGROK_AUTHTOKEN')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [8]:
# whisper 모델 설치
!pip install git+https://github.com/openai/whisper.git -q

# Whisper 모델 로드 (Medium 사이즈)
import whisper
model = whisper.load_model("medium")

# Youtube MP3 추출에 필요한 라이브러리
!pip install -U yt-dlp

# Flask 서버에 필요한 라이브러리 설치 및 토큰 설정
!pip install flask flask-ngrok pyngrok flask-cors
!ngrok authtoken {ngrok_token}

# gpt api를 위한 라이브러리
!pip install openai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
model.device

device(type='cuda', index=0)

# Process code on Flask

In [10]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import yt_dlp
import whisper
import shutil
import os
import re
from flask_cors import CORS

import openai
import time

app = Flask(__name__)
CORS(app)

public_url = ngrok.connect(5000)
print("ngrok tunnel available at:", public_url)

def sanitize_filename(filename):
    """파일 이름에서 허용되지 않는 문자를 제거합니다."""
    return re.sub(r'[\\/*?:"<>|\']', "", filename)

def download_video_as_mp3(url, custom_name):
    safe_custom_name = sanitize_filename(custom_name)
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': f"/content/drive/My Drive/vidie-audie/contents/{safe_custom_name}.%(ext)s",
        'windowsfilenames': False  # 윈도우 파일 이름 규칙을 따르지 않도록 설정 #다루고 있는 safe_custom_name과 실제 파일명이 달라질 수 있는 잠재적 가능성 방지
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
        return safe_custom_name


client = openai.OpenAI(
    api_key=openai_api_key
)

SUMMARY_PROMPT = """입력되는 텍스트를 500자 내외로 요약해주고, 해당 텍스트의 핵심 키워드 단어를 하나 골라서 요약문과 같이 제공해줘
너가 제공해줘야 할 출력 형태는 '키워드 단어 & 500자 내외의 요약문'이야. 출력 형태를 반드시 지켜"""

def read_text_file(file_path) :
  with open(file_path, 'r', encoding='utf-8') as file:
    return file.read()

def summarize_text(text) :
  max_retries = 5
  for i in range(max_retries) :
      try :
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role" : "system", "content" : SUMMARY_PROMPT},
                {"role": "user", "content" : text}
            ],
            temperature=0.5 # 0.0: 결정적, 1.0: 창의적
        )
        message_content = response.choices[0].message.content.strip()
        if '&' in message_content :
          return message_content
        else :
          print("응답이 예상한 것과 다르게 나왔습니다. 다시 시도합니다.")

      except openai.RateLimitError:
        print(f"RateLimitError 발생 {2 ** i}초 뒤 재호출하겠습니다.")
        time.sleep(2 ** i)
      except openai.APIError as e :
        print(f"OpenAIError 발생 : {str(e)}")
        raise e

  raise Exception("error : 최대 재시도 횟수 초과")


@app.route("/")
def hello():
    return "Hello, World!"

@app.route("/test/")
def test() :
    return "This is test page"

@app.route("/process", methods=["POST"])
def process() :
  data = request.json
  down_url = data['url']
  custom_name = data['name']

  title = download_video_as_mp3(down_url, custom_name)
  print("mp3 다운로드 완료!")
  mp3_file_path = f"/content/drive/My Drive/vidie-audie/contents/{title}.mp3"

  result = model.transcribe(mp3_file_path)

  txt_file_path = f"/content/drive/My Drive/vidie-audie/contents/{title}.txt"

  with open(txt_file_path, 'w', encoding='utf-8') as file:
      file.write(result['text'])
  print(f"텍스트 파일 저장 완료! 파일 경로: {txt_file_path}")

  input_file_path = txt_file_path
  text = read_text_file(input_file_path)
  text_output = summarize_text(text).split(' & ')

  keyword = text_output[0]
  summary = text_output[1]

  return jsonify({"message": "Processing complete", "mp3_file_path": mp3_file_path, "txt_file_path": txt_file_path, "keyword": keyword, "summary": summary})
  print('response complete')

app.run(port=5000)

ngrok tunnel available at: NgrokTunnel: "https://2424-34-142-236-166.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Jun/2024 10:48:46] "OPTIONS /process HTTP/1.1" 200 -


[youtube] Extracting URL: https://youtu.be/y5BzqkH8JxE?si=qdQLMLY6bDJns1ac
[youtube] y5BzqkH8JxE: Downloading webpage


[youtube] y5BzqkH8JxE: Downloading ios player API JSON


[youtube] y5BzqkH8JxE: Downloading iframe API JS


[youtube] y5BzqkH8JxE: Downloading web player API JSON


ERROR: [youtube] y5BzqkH8JxE: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
ERROR:__main__:Exception on /process [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/YoutubeDL.py", line 1606, in wrapper
    return func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/YoutubeDL.py", line 1741, in __extract_info
    ie_result = ie.extract(url)
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/extractor/common.py", line 734, in extract
    ie_result = self._real_extract(url)
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/extractor/youtube.py", line 4145, in _real_extract
    webpage, master_ytcfg, player_responses, player_url = self._download_player_responses(url, smuggled_data, video_id, webpage_url)
  File "/usr/local/lib/python3.10/dist-p